In [1]:
pip install gensim

#Importing the gensim library


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import the required libraries 
import nltk 
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
import numpy as np 
import gensim
from gensim.models import Word2Vec
import pandas as pd 

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abcd\AppData\Roaming\nltk_data...


True

In [6]:
df_data = pd.read_csv("C:/Users/HP\CP/Manmohan/PM_Manmohan_SpeechCleaned (1).csv")

In [11]:
df_data

,Date,Location,Title,Speech,Cleaned_Speech,Cleaned_Speech_string
0,"May 17, 2014",New Delhi,PM's address to the Nation,"\n\tMy Fellow Citizens,\n\tI address you today...",['fellow citizen address today last time prime...,fellow citizen address today last time prime m...
1,"March 4, 2014","Nay Pyi Taw, Myanmar",PM’s statement at 3rd BIMSTEC Summit,\n\t“I am delighted to return to this beautifu...,['delighted return beautiful city nay pyi taw ...,delighted return beautiful city nay pyi taw th...
2,"March 1, 2014",New Delhi,PM’s address at the National Awards Ceremony f...,\n\t“I am very happy to participate in the Nat...,['happy participate national award function mi...,happy participate national award function mini...
3,"February 26, 2014",New Delhi,PM's speech at the foundation laying ceremony ...,\n\t“Today marks a very important step forward...,['today mark important step forward towards es...,today mark important step forward towards esta...
4,"February 21, 2014",New Delhi,PM’s farewell speech to the 15th Lok Sabha,"""Madam Speaker,\n\nAs we come to the end of th...",['madam speaker come end journey fifteenth lok...,madam speaker come end journey fifteenth lok s...
...,...,...,...,...,...,...
1396,"July 31, 2004",Bangkok,Speech by the Prime Minister of India at the i...,"\n\t\t \n\tMr. Chairman,\n\t \n\tExcellencies,...",['mr chairman excellency lady gentleman begin ...,mr chairman excellency lady gentleman begin co...
1397,"July 26, 2004",New Delhi,PM's speech at the CSIR Society Meeting,"Distinguished Members of the CSIR Society, La...",['distinguished member csir society lady gentl...,distinguished member csir society lady gentlem...
1398,"July 3, 2004",New Delhi,"PM's inaugural address at the ""Dialogue on Min...","""It gives me great pleasure to be amongst you...","['give great pleasure amongst morning', 'parti...",give great pleasure amongst morning particular...
1399,"June 29, 2004",New Delhi,PM's inaugural address at the Chief Ministers'...,"""I welcome you all to this Conference which w...",['welcome conference take stock effort rural d...,welcome conference take stock effort rural dev...


In order to sort the dates, firstly changing the format of the date. The code below defines a function to convert the date strings in the Date column of a Pandas DataFrame to a datetime format. 

In [17]:

from datetime import datetime
month_dict = {
    "January": "01",
    "February": "02",
    "March": "03",
    "April": "04",
    "May": "05",
    "June": "06",
    "July": "07",
    "August": "08",
    "September": "09",
    "October": "10",
    "November": "11",
    "December": "12"
}

def dates(x):
    dt = datetime.strptime(x, "%B %d, %Y")
    return dt
# It then applies this function to the Date column and sorts the DataFrame by date in ascending order. 
 
df_data['Date'] = df_data['Date'].apply(dates)
df_data = df_data.sort_values('Date')


In [18]:
df_data

,Date,Location,Title,Speech,Cleaned_Speech,Cleaned_Speech_string
1400,2004-06-24,New Delhi,Prime Minister 's Address to the Nation,"My Fellow Citizens,I speak to you with a deep...",['fellow citizen speak deep sense humility ful...,fellow citizen speak deep sense humility fully...
1399,2004-06-29,New Delhi,PM's inaugural address at the Chief Ministers'...,"""I welcome you all to this Conference which w...",['welcome conference take stock effort rural d...,welcome conference take stock effort rural dev...
1398,2004-07-03,New Delhi,"PM's inaugural address at the ""Dialogue on Min...","""It gives me great pleasure to be amongst you...","['give great pleasure amongst morning', 'parti...",give great pleasure amongst morning particular...
1397,2004-07-26,New Delhi,PM's speech at the CSIR Society Meeting,"Distinguished Members of the CSIR Society, La...",['distinguished member csir society lady gentl...,distinguished member csir society lady gentlem...
1395,2004-07-31,Bangkok,PM’ statement at the Press Conference,"\n\t""My two days in Bangkok have been very use...","['two day bangkok useful productive', 'first b...",two day bangkok useful productive first bimste...
...,...,...,...,...,...,...
4,2014-02-21,New Delhi,PM’s farewell speech to the 15th Lok Sabha,"""Madam Speaker,\n\nAs we come to the end of th...",['madam speaker come end journey fifteenth lok...,madam speaker come end journey fifteenth lok s...
3,2014-02-26,New Delhi,PM's speech at the foundation laying ceremony ...,\n\t“Today marks a very important step forward...,['today mark important step forward towards es...,today mark important step forward towards esta...
2,2014-03-01,New Delhi,PM’s address at the National Awards Ceremony f...,\n\t“I am very happy to participate in the Nat...,['happy participate national award function mi...,happy participate national award function mini...
1,2014-03-04,"Nay Pyi Taw, Myanmar",PM’s statement at 3rd BIMSTEC Summit,\n\t“I am delighted to return to this beautifu...,['delighted return beautiful city nay pyi taw ...,delighted return beautiful city nay pyi taw th...


### INDEXING 

In [19]:
#changing the index
new_index = ['MSS' + str(i) for i in range(1, len(df_data)+1)]
#renaming to reflect on df
df_data = df_data.rename(index=dict(zip(df_data.index, new_index)))
df_data

,Date,Location,Title,Speech,Cleaned_Speech,Cleaned_Speech_string
MSS1,2004-06-24,New Delhi,Prime Minister 's Address to the Nation,"My Fellow Citizens,I speak to you with a deep...",['fellow citizen speak deep sense humility ful...,fellow citizen speak deep sense humility fully...
MSS2,2004-06-29,New Delhi,PM's inaugural address at the Chief Ministers'...,"""I welcome you all to this Conference which w...",['welcome conference take stock effort rural d...,welcome conference take stock effort rural dev...
MSS3,2004-07-03,New Delhi,"PM's inaugural address at the ""Dialogue on Min...","""It gives me great pleasure to be amongst you...","['give great pleasure amongst morning', 'parti...",give great pleasure amongst morning particular...
MSS4,2004-07-26,New Delhi,PM's speech at the CSIR Society Meeting,"Distinguished Members of the CSIR Society, La...",['distinguished member csir society lady gentl...,distinguished member csir society lady gentlem...
MSS5,2004-07-31,Bangkok,PM’ statement at the Press Conference,"\n\t""My two days in Bangkok have been very use...","['two day bangkok useful productive', 'first b...",two day bangkok useful productive first bimste...
...,...,...,...,...,...,...
MSS1397,2014-02-21,New Delhi,PM’s farewell speech to the 15th Lok Sabha,"""Madam Speaker,\n\nAs we come to the end of th...",['madam speaker come end journey fifteenth lok...,madam speaker come end journey fifteenth lok s...
MSS1398,2014-02-26,New Delhi,PM's speech at the foundation laying ceremony ...,\n\t“Today marks a very important step forward...,['today mark important step forward towards es...,today mark important step forward towards esta...
MSS1399,2014-03-01,New Delhi,PM’s address at the National Awards Ceremony f...,\n\t“I am very happy to participate in the Nat...,['happy participate national award function mi...,happy participate national award function mini...
MSS1400,2014-03-04,"Nay Pyi Taw, Myanmar",PM’s statement at 3rd BIMSTEC Summit,\n\t“I am delighted to return to this beautifu...,['delighted return beautiful city nay pyi taw ...,delighted return beautiful city nay pyi taw th...


In [20]:
# Converts the Speech column to a string data type.
df_data['Speech'] = df_data['Speech'].astype(str)

In [21]:
# Initializing the model 
lemmatizer = WordNetLemmatizer()

Now we define a function to clean the speeches. Cleaning involves tokenization, lemmitization removing special characters and removing stopwords. 

In [22]:
def cleaned(speech):
    # Sentence Tokenization
    sentence = nltk.sent_tokenize(speech)

    # Creating an empty list to store the cleaned dataset 
    corpus = []

    # Loop through each sentence in the speech
    for i in range(len(sentence)):
        # Remove any non-alphabetic characters in the sentence and convert to lowercase
        review = re.sub("[^a-zA-Z0-9]", " ", sentence[i])
        review = review.lower()
        
        # Tokenize the sentence into a list of words
        review = review.split()
        
        # Lemmatize each word in the sentence and remove stop words
        lemmatizer = WordNetLemmatizer()
        review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words("english"))]
        
        # Convert the list of words back into a sentence
        review = ' '.join(review)
        
        # Add the cleaned sentence to the corpus
        corpus.append(review)

    return corpus 

In [23]:
# To convert cleaned speeches into string for performing NER
def string(speech):
    string_speech = " ".join(speech)
    return string_speech

In [24]:
# Clean the text data in the 'Speech' column of the dataframe using the 'cleaned' function
# and create a new column 'Cleaned_Speech' to store the cleaned text data
df_data['Cleaned_Speech'] = df_data['Speech'].apply(cleaned)

# Convert the cleaned text data in the 'Cleaned_Speech' column to a string 
# using the 'string' function and create a new column 'Cleaned_Speech_string'
# to store the string data
df_data['Cleaned_Speech_string'] = df_data['Cleaned_Speech'].apply(string)

In [25]:
df_data = df_data.reset_index()

In [26]:
df_data

,index,Date,Location,Title,Speech,Cleaned_Speech,Cleaned_Speech_string
0,MSS1,2004-06-24,New Delhi,Prime Minister 's Address to the Nation,"My Fellow Citizens,I speak to you with a deep...",[fellow citizen speak deep sense humility full...,fellow citizen speak deep sense humility fully...
1,MSS2,2004-06-29,New Delhi,PM's inaugural address at the Chief Ministers'...,"""I welcome you all to this Conference which w...",[welcome conference take stock effort rural de...,welcome conference take stock effort rural dev...
2,MSS3,2004-07-03,New Delhi,"PM's inaugural address at the ""Dialogue on Min...","""It gives me great pleasure to be amongst you...","[give great pleasure amongst morning, particul...",give great pleasure amongst morning particular...
3,MSS4,2004-07-26,New Delhi,PM's speech at the CSIR Society Meeting,"Distinguished Members of the CSIR Society, La...",[distinguished member csir society lady gentle...,distinguished member csir society lady gentlem...
4,MSS5,2004-07-31,Bangkok,PM’ statement at the Press Conference,"\n\t""My two days in Bangkok have been very use...","[two day bangkok useful productive, first bims...",two day bangkok useful productive first bimste...
...,...,...,...,...,...,...,...
1396,MSS1397,2014-02-21,New Delhi,PM’s farewell speech to the 15th Lok Sabha,"""Madam Speaker,\n\nAs we come to the end of th...",[madam speaker come end journey fifteenth lok ...,madam speaker come end journey fifteenth lok s...
1397,MSS1398,2014-02-26,New Delhi,PM's speech at the foundation laying ceremony ...,\n\t“Today marks a very important step forward...,[today mark important step forward towards est...,today mark important step forward towards esta...
1398,MSS1399,2014-03-01,New Delhi,PM’s address at the National Awards Ceremony f...,\n\t“I am very happy to participate in the Nat...,[happy participate national award function min...,happy participate national award function mini...
1399,MSS1400,2014-03-04,"Nay Pyi Taw, Myanmar",PM’s statement at 3rd BIMSTEC Summit,\n\t“I am delighted to return to this beautifu...,[delighted return beautiful city nay pyi taw t...,delighted return beautiful city nay pyi taw th...


In [27]:
# Saving the cleaned data to the csv file 
df_data.to_csv("C:/Users/HP/Downloads/PM_MANMOHAN_Speech_Cleaned_string (1).csv", index = False)